## Load data and creating set-up

In [ ]:
import os
import mne
import pyxdf
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from joblib import dump, load
import pandas as pd
from sklearn.decomposition import FastICA
import joblib
import mne

In [ ]:
fn = os.path.join(os.path.expanduser("~"), "Desktop", "Radboud", "work", "BCI", "cVEP", "sub-VPpdia", "ses-S001", "eeg", 
                  "sub-VPpdia_ses-S001_task-covert_run-001_eeg.xdf")
streams = pyxdf.load_xdf(fn)[0]
names = [stream["info"]["name"][0] for stream in streams]
stream_id = names.index("BioSemi")
stream = streams[stream_id]

sfreq = float(stream["info"]["nominal_srate"][0])
montage = mne.channels.make_standard_montage("biosemi64")  # Add channel information

info = mne.create_info(ch_names=montage.ch_names, sfreq=sfreq, ch_types=64 * ["eeg"], verbose=False)
data = stream["time_series"][:, 1:65]  # Select EEG only

raw = mne.io.RawArray(data.T, info, verbose=False)
raw._filenames = [fn]
raw.set_montage(montage);

### Pre-defined functions
Below are functions that are used later on in the notebook.

In [ ]:
def plot_ica_analysis(ica, raw, start_time, end_time):
    sfreq = raw.info['sfreq']
    start_idx = int(start_time * sfreq)
    end_idx = int(end_time * sfreq)
    for component_idx in range(ica.n_components_):
        fig, ax = plt.subplots(2, 2, figsize=(10, 8), gridspec_kw={'width_ratios': [1, 2], 'height_ratios': [2, 1]})
        fig.subplots_adjust(wspace=0.4, hspace=0.4)
        fig.suptitle(f"ICA Component {component_idx} Analysis", fontsize=16)

        # Topography (W)
        mne.viz.plot_ica_components(ica, picks=[component_idx], ch_type='eeg', axes=ax[0, 0], colorbar=False, show=False)
        ax[0, 0].set_title(f"Component {component_idx} Topography")

        # Time Series (S)
        ica_sources = ica.get_sources(raw).get_data()[component_idx]
        times_focus = raw.times[start_idx:end_idx]
        ica_sources_focus = ica_sources[start_idx:end_idx]
        ax[0, 1].plot(times_focus, ica_sources_focus, lw=1)
        ax[0, 1].set_title(f"Time Series of Component {component_idx} ({start_time}-{end_time}s)")
        ax[0, 1].set_xlabel('Time (s)')
        ax[0, 1].set_ylabel('Amplitude')

        # Frequency Spectrum
        freqs = np.fft.rfftfreq(n=len(ica_sources_focus), d=1/sfreq)
        fft_vals = np.abs(fft(ica_sources_focus))[:len(freqs)]
        max_freq = 100
        freq_mask = freqs <= max_freq
        power_spectrum_dB = 10 * np.log10(fft_vals[freq_mask] ** 2) 
        ax[1, 1].plot(freqs[freq_mask], power_spectrum_dB, lw=1)
        ax[1, 1].set_title(f"Frequency Spectrum of Component {component_idx} (Max 100 Hz)")
        ax[1, 1].set_xlabel('Frequency (Hz)')
        ax[1, 1].set_ylabel('Magnitude (dB)')
        ax[1, 0].axis('off')
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()
        
def plot_ica_analysis_individual(ica, raw, component_idx, start_time, end_time):
    sfreq = raw.info['sfreq']
    start_idx = int(start_time * sfreq)
    end_idx = int(end_time * sfreq)

    fig, ax = plt.subplots(2, 2, figsize=(10, 8), gridspec_kw={'width_ratios': [1, 2], 'height_ratios': [2, 1]})
    fig.subplots_adjust(wspace=0.4, hspace=0.4)
    fig.suptitle(f"ICA Component {component_idx} Analysis", fontsize=16)

    # Topography (W)
    mne.viz.plot_ica_components(ica, picks=[component_idx], ch_type='eeg', axes=ax[0, 0], colorbar=False, show=False)
    ax[0, 0].set_title(f"Component {component_idx} Topography")

    # Time Series (S)
    ica_sources = ica.get_sources(raw).get_data()[component_idx]
    times_focus = raw.times[start_idx:end_idx]
    ica_sources_focus = ica_sources[start_idx:end_idx]
    ax[0, 1].plot(times_focus, ica_sources_focus, lw=1)
    ax[0, 1].set_title(f"Time Series of Component {component_idx} ({start_time}-{end_time}s)")
    ax[0, 1].set_xlabel('Time (s)')
    ax[0, 1].set_ylabel('Amplitude')

    # Frequency Spectrum
    freqs = np.fft.rfftfreq(n=len(ica_sources_focus), d=1/sfreq)
    fft_vals = np.abs(fft(ica_sources_focus))[:len(freqs)]
    max_freq = 100
    freq_mask = freqs <= max_freq
    power_spectrum_dB = 10 * np.log10(fft_vals[freq_mask] ** 2)  # Squaring FFT values for power calculation
    ax[1, 1].plot(freqs[freq_mask], power_spectrum_dB, lw=1)
    ax[1, 1].set_title(f"Frequency Spectrum of Component {component_idx} (Max 100 Hz)")
    ax[1, 1].set_xlabel('Frequency (Hz)')
    ax[1, 1].set_ylabel('Magnitude (dB)')
    ax[1, 0].axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

### Delete bad electrodes
The electrodes are deleted based on the previous round I did. For every participant I noted down what electrodes should be excluded and for what reason. Because of this, no in-depth analysis is performed for all electrodes in this notebook.

In [ ]:
channels_to_drop = ['']
raw.drop_channels(channels_to_drop)

### Create log files and keep up with them

In [ ]:
participants = ['VPpdizc']


# Define the base directory
base_dir = "/Users/juliette/Desktop/participant_files" 

# Create the main directory if it doesn't exist
os.makedirs(base_dir, exist_ok=True)

# Create log files with structured CSV format
for participant in participants:
    participant_folder = os.path.join(base_dir, participant)  
    os.makedirs(participant_folder, exist_ok=True)  

    # Define file paths
    bad_elec_file = os.path.join(participant_folder, f"sub-{participant}_bad_elec.csv")
    deleted_eye_file = os.path.join(participant_folder, f"sub-{participant}_deleted_eye_components.csv")
    retained_noise_file = os.path.join(participant_folder, f"sub-{participant}_retained_noise_components.csv")

    # Write bad electrodes file
    with open(bad_elec_file, "w") as file:
        file.write("Electrode,Reason\n")

    # Write deleted eye components file
    with open(deleted_eye_file, "w") as file:
        file.write("Component,Reason\n")
        
    # Write retained noise components file
    with open(retained_noise_file, "w") as file:
        file.write("Component,Reason\n")


In [ ]:
# Define the file path for a specific participant
participant = "VPpdizc"
file_path = f"/Users/juliette/Desktop/participant_files/{participant}/sub-{participant}_bad_elec.csv"

# Load the existing file
df = pd.read_csv(file_path)

# Add a new electrode to the file
new_entry = pd.DataFrame({
    "Electrode": ["P1", 'PO3', 'POz', "Pz", 'P2', "FC2"], 
    "Reason": ["Reported as a bad electrode during the experiment.",
              "Reported as a bad electrode during the experiment.",
              "Reported as a bad electrode during the experiment.",
              "Reported as a bad electrode during the experiment.",
              "Reported as a bad electrode during the experiment.",
              "Large deflections."]
})
  
df = pd.concat([df, new_entry], ignore_index=True)

# Save back to CSV
df.to_csv(file_path, index=False)

print(f"Updated {file_path}")


### Creating the ICA models
Here I fit the ICA for the participants after some preprosessing steps.

In [ ]:
save_dir = os.path.expanduser("~") + "/Desktop/participant_files" 

for participant in participants:
    # Load the EEG data
    fn = os.path.join(os.path.expanduser("~"), "Desktop", "Radboud", "work", "BCI", "cVEP", 
                      f"sub-{participant}", "ses-S001", "eeg", 
                      f"sub-{participant}_ses-S001_task-covert_run-001_eeg.xdf")

    # Load the XDF file
    streams = pyxdf.load_xdf(fn)[0]
    names = [stream["info"]["name"][0] for stream in streams]
    stream_id = names.index("BioSemi")
    stream = streams[stream_id]

    # Extract sampling rate and data
    sfreq = float(stream["info"]["nominal_srate"][0])
    montage = mne.channels.make_standard_montage("biosemi64")  # Add channel information

    info = mne.create_info(ch_names=montage.ch_names, sfreq=sfreq, ch_types=64 * ["eeg"], verbose=False)
    data = stream["time_series"][:, 1:65]  # Select EEG data (first 64 channels)

    # Create raw MNE object
    raw = mne.io.RawArray(data.T, info, verbose=False)
    raw._filenames = [fn]
    raw.set_montage(montage)
    
    # Apply a spectral filter
    raw.load_data()
    raw.filter(l_freq=1.0, h_freq=100, verbose=False)  # Apply the 1-100 Hz bandpass filter
    raw.notch_filter(freqs=np.arange(50, raw.info['sfreq'] / 2, 50))  # Apply notch filter

    # Read bad electrode list from the CSV file
    bad_elec_file = os.path.join(save_dir, participant, f"sub-{participant}_bad_elec.csv")  # CSV file
    bad_electrodes = []

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(bad_elec_file)
    bad_electrodes = df["Electrode"].dropna().tolist()
    
    # Drop the bad electrodes from the raw data
    raw.drop_channels(bad_electrodes)
    
    print(bad_electrodes)

    # Learn ICA
    ica = mne.preprocessing.ICA(random_state=42)
    ica.fit(raw)

    # Create participant's folder if it doesn't exist
    participant_folder = os.path.join(save_dir, participant)
    os.makedirs(participant_folder, exist_ok=True)

    # Save the ICA model in the participant's folder
    ica_filename = os.path.join(participant_folder, f"{participant}_ICA_.joblib")
    joblib.dump(ica, ica_filename)

    print(f"ICA model for {participant} saved to {ica_filename}")


## Plotting the ICA and analysing for removal
In this section all components are plotted and removed when they appear to be eyeblinks, or prominent noise. The EEG data is first loaded in the same way as is done in the previous cell. Then, the ICA model is loaded and the components are plotted.

In [ ]:
# Load the ICA model
participant = "VPpdizc"
ICA = joblib.load(f'/Users/juliette/Desktop/participant_files/{participant}/{participant}_ICA_.joblib')

# Load EEG data
fn = os.path.join(os.path.expanduser("~"), "Desktop", "Radboud", "work", "BCI", "cVEP", 
                  f"sub-{participant}", "ses-S001", "eeg", 
                  f"sub-{participant}_ses-S001_task-covert_run-001_eeg.xdf")

# Load the XDF file
streams = pyxdf.load_xdf(fn)[0]
names = [stream["info"]["name"][0] for stream in streams]
stream_id = names.index("BioSemi")
stream = streams[stream_id]

# Extract EEG data and sampling rate
sfreq = float(stream["info"]["nominal_srate"][0])
montage = mne.channels.make_standard_montage("biosemi64")

info = mne.create_info(ch_names=montage.ch_names, sfreq=sfreq, ch_types=64 * ["eeg"], verbose=False)
data = stream["time_series"][:, 1:65]  # Select EEG data (first 64 channels)

raw = mne.io.RawArray(data.T, info, verbose=False)
raw.set_montage(montage)

# Apply filtering (same as before)
raw.load_data()
raw.filter(l_freq=1.0, h_freq=100, verbose=False)
raw.notch_filter(freqs=np.arange(50, raw.info['sfreq'] / 2, 50))

# Filter out rows where the Electrode is "-" (no electrode to remove)
bad_electrodes = df["Electrode"].dropna().tolist()

# Drop the bad electrodes from the raw data
raw.drop_channels(bad_electrodes)

In [ ]:
# Load the saved ICA model
ica_filename = os.path.join(save_dir, participant, f"{participant}_ICA_.joblib")
ica = joblib.load(ica_filename)


# Plot the ICA components
n_components = ica.n_components_
chunk_size = 16

for i in range(0, n_components, chunk_size):
    picks = np.arange(i, min(i + chunk_size, n_components))
    
    # Plot the sources for the selected component range
    ica.plot_sources(raw, picks=picks, show_scrollbars=False, start=400, stop=500)

In [ ]:
ica.plot_components();

In [ ]:
start = 0
end = 100
plot_ica_analysis(ica, raw, start, end)

In [ ]:
components = [16, 38, 44, 49, 57, 12, 10, 11, 31, 20, 21, 22, 23, 33]
for i in components:
    plot_ica_analysis_individual(ica, raw, i, 400, 500)

### Keeping up log for artefacts
Keeping up the log for the eyeblinks.

In [ ]:
# File path to update component log (eyeblinks)
file_path = f"/Users/juliette/Desktop/participant_files/{participant}/sub-{participant}_deleted_eye_components.csv"

# Load the existing file
df = pd.read_csv(file_path)

# Add a new electrode to the file
new_entry = pd.DataFrame({
    "Component": ["0", "3", "4", "7", "13"], 
    "Reason": ["Appear to be eyeblinks due to peaks in time series and frontal activity.",
               "Appear to be eyeblinks due to peaks in time series and frontal activity.",
               "Appear to be eyeblinks due to peaks in time series and frontal activity.",
               "Appear to be eyeblinks due to peaks in time series and frontal activity.",
               "Appear to be eyeblinks due to peaks in time series and frontal activity."
              ]
})
  
df = pd.concat([df, new_entry], ignore_index=True)

# Save back to CSV
df.to_csv(file_path, index=False)

print(f"Updated {file_path}")


Keeping up the log for the noisy components

In [ ]:
# File path to update component log (noise)
file_path = f"/Users/juliette/Desktop/participant_files/{participant}/sub-{participant}_retained_noise_components.csv"

# Load the existing file
df = pd.read_csv(file_path)

# Add a new electrode to the file
new_entry = pd.DataFrame({
    "Component": ["16", "38", "44", "12", "10", "21", "23", "20", "31", "21"], 
    "Reason": ["Noisy time series.",
               "Noisy time series.",
               "Noisy time series.",
               "Noisy time series.",
               "Noisy time series.",
               "Noisy time series.",
               "Noisy time series.",
               "Noisy time series.",
               "Noisy time series.",
               "Noisy time series."
               
              ]})
  
df = pd.concat([df, new_entry], ignore_index=True)

# Save back to CSV
df.to_csv(file_path, index=False)

print(f"Updated {file_path}")

In [ ]:
ica.exclude = [0, 3, 4, 7, 13]
ica.apply(raw)